### **환경설정**

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import scikit_posthocs as sp
from pingouin import welch_anova
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# **진행 과정**

* 첫 번째 분석에서 결과 해석의 오류가 있어 총 두 번에 걸쳐 분석을 진행하였다.

## **첫번째 시도**

### **데이터 불러오기 및 EDA**

In [ ]:
# 데이터 불러오기

complete_episode_day_interval = pd.read_csv('./src/user_segment_complete_episode_interval.csv')

In [3]:
# 데이터 확인

complete_episode_day_interval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10177 entries, 0 to 10176
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   user_id                         10177 non-null  object 
 1   segmentation                    10177 non-null  object 
 2   avg_complete_episode_intervals  10177 non-null  float64
dtypes: float64(1), object(2)
memory usage: 238.7+ KB


* 데이터프레임에 총 10,177개 행이 잘 들어있고, 결측치가 존재하지 않는다.

In [4]:
# 사용자 세그먼트별로 각 사용자의 평균 complete.episode day interval들을 Series로 저장

complete_episode_day_interval_core = complete_episode_day_interval.loc[complete_episode_day_interval['segmentation']=='핵심', 'avg_complete_episode_intervals']
complete_episode_day_interval_temp = complete_episode_day_interval.loc[complete_episode_day_interval['segmentation']=='일시적', 'avg_complete_episode_intervals']
complete_episode_day_interval_cold = complete_episode_day_interval.loc[complete_episode_day_interval['segmentation']=='냉담', 'avg_complete_episode_intervals']

In [5]:
# 사용자 세그먼트별 평균 complete.episode day interval

print("[사용자 세그먼트별 평균 complete.episode day interval]\n")
print(f"핵심 사용자: {round(np.mean(complete_episode_day_interval_core), 2)}일")
print(f"일시적 사용자: {round(np.mean(complete_episode_day_interval_temp), 2)}일")
print(f"냉담 사용자: {round(np.mean(complete_episode_day_interval_cold), 2)}일")

[사용자 세그먼트별 평균 complete.episode day interval]

핵심 사용자: 4.1일
일시적 사용자: 7.82일
냉담 사용자: 17.35일


* 우리는 사용자 세그먼트별 평균 `complete.episode` day interval의 차이가 유의미하게 있는지를 알아보기 위해 ANOVA를 진행하려 한다.

In [6]:
# 사용자 세그먼트별 사용자 수

print("[사용자 세그먼트별 사용자 수]\n")
print(f"핵심 사용자: {complete_episode_day_interval_core.size}명")
print(f"일시적 사용자: {complete_episode_day_interval_temp.size}명")
print(f"냉담 사용자: {complete_episode_day_interval_cold.size}명")

[사용자 세그먼트별 사용자 수]

핵심 사용자: 1200명
일시적 사용자: 5602명
냉담 사용자: 3375명


### **ANOVA 진행 전 데이터 사전 검정**

* ANOVA를 진행하기 전 데이터에 대한 사전 검정이 필요하다.
* 첫째로 각 세그먼트의 데이터가 정규성을 만족하고, 둘째로 세 세그먼트의 데이터가 등분산성을 만족해야 ANOVA를 진행할 수 있다.

#### **1. 정규성 검정**

* 사용자 세그먼트별 사용자 수를 확인하면 핵심 사용자와 냉담 사용자는 5000명 미만이고, 일시적 사용자는 5000명 이상인 것을 확인할 수 있다.
* 따라서 핵심 사용자와 냉담 사용자는 Shapiro-Wilk test로, 일시적 사용자는 Anderson-Darling test로 정규성 검정을 진행한다.

In [7]:
# 핵심 사용자 세그먼트 정규성 검정

print(stats.shapiro(complete_episode_day_interval_core))

ShapiroResult(statistic=np.float64(0.7803049517332192), pvalue=np.float64(2.397530332328133e-37))


* p-value가 2.397530332328133e-37로 매우 작으므로, 핵심 사용자 세그먼트의 데이터는 정규성을 만족하지 못한다고 할 수 있다.

In [8]:
# 일시적 사용자 세그먼트 정규성 검정

print(stats.anderson(complete_episode_day_interval_temp, dist='norm'))

AndersonResult(statistic=np.float64(520.4958730339786), critical_values=array([0.576, 0.656, 0.786, 0.917, 1.091]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=np.float64(7.815574260149581), scale=np.float64(9.405563557876654))
 success: True
 message: '`anderson` successfully fit the distribution to the data.')


* 검정통계량 값은 520.4958730339786로, 유의수준이 1%일 때의 임계값인 1.091보다도 매우 크다.
* 따라서 일시적 사용자 세그먼트의 데이터가 정규분포에서 추출되었다는 귀무가설은 기각된다.
* 즉 일시적 사용자 세그먼트의 데이터는 정규성을 만족하지 못한다고 할 수 있다.

In [9]:
# 냉담 사용자 세그먼트 정규성 검정

print(stats.shapiro(complete_episode_day_interval_cold))

ShapiroResult(statistic=np.float64(0.41372427036381765), pvalue=np.float64(3.5025161259193495e-74))


* p-value가 3.5025161259193495e-74로 매우 작으므로, 냉담 사용자 세그먼트의 데이터는 정규성을 만족하지 못한다고 할 수 있다.

* test 결과, 사용자 세그먼트별로 각 사용자의 평균 `complete.episode` day interval들은 **정규성을 만족하지 못한다**.
* 하지만 위에서 사용한 test는 귀무가설이 채택되기 상당히 까다로운 test이다.
* 이로 인해 **실무에서는 왜도의 절대값이 2보다 작고, 첨도가 7보다 작으면 정규성을 만족한다**고 본다. (표본 수가 300 이상일 경우)  
참고: [Statistical notes for clinical researchers: assessing normal distribution (2) using skewness and kurtosis](https://pmc.ncbi.nlm.nih.gov/articles/PMC3591587/)

##### **Practical Approach: 왜도와 첨도 이용**

In [10]:
# 사용자 세그먼트별 데이터의 왜도의 절대값 계산

print("[사용자 세그먼트별 데이터의 왜도의 절대값]\n")
print(f"핵심 사용자: {round(abs(stats.skew(complete_episode_day_interval_core, bias=False)), 2)}")
print(f"일시적 사용자: {round(abs(stats.skew(complete_episode_day_interval_temp, bias=False)), 2)}")
print(f"냉담 사용자: {round(abs(stats.skew(complete_episode_day_interval_cold, bias=False)), 2)}")

[사용자 세그먼트별 데이터의 왜도의 절대값]

핵심 사용자: 2.44
일시적 사용자: 4.11
냉담 사용자: 6.32


* 왜도의 절대값이 모두 2보다 크다.

In [11]:
# (왜도의 절대값이 모두 2보다 크면 무용지물이지만) 사용자 세그먼트별 데이터의 첨도 계산

print("[사용자 세그먼트별 데이터의 첨도]\n")
print(f"핵심 사용자: {round(stats.kurtosis(complete_episode_day_interval_core, fisher=True), 2)}")
print(f"일시적 사용자: {round(stats.kurtosis(complete_episode_day_interval_temp, fisher=True), 2)}")
print(f"냉담 사용자: {round(stats.kurtosis(complete_episode_day_interval_cold, fisher=True), 2)}")

[사용자 세그먼트별 데이터의 첨도]

핵심 사용자: 9.31
일시적 사용자: 29.69
냉담 사용자: 58.88


* 첨도가 모두 7보다 크다.

* 즉 왜도와 첨도를 이용해도 사용자 세그먼트별 데이터들은 정규성을 만족하지 못한다.
* 따라서 ANOVA를 진행할 수 없으므로 Kruskal-Wallis H Test를 진행한다.

### **Kruskal-Wallis H Test**

* $H_0$: 모든 집단의 중위수가 같다.
* $H_1$: 한 집단이라도 중위수가 같지 않다.

In [12]:
# Kruskal-Wallis H Test

print(stats.kruskal(complete_episode_day_interval_core, complete_episode_day_interval_temp, complete_episode_day_interval_cold))

KruskalResult(statistic=np.float64(191.15968865261186), pvalue=np.float64(3.091726211560913e-42))


* p-value가 3.091726211560913e-42로 매우 작으므로 귀무가설은 기각된다.
* 즉 한 사용자 세그먼트라도 중위수가 같지 않다고 할 수 있다.

### **다중비교(사후분석)**

* 우리는 모든 사용자 세그먼트의 중위수가 각각 유의미한 차이를 가지는지 알고 싶기 때문에 다중비교(사후분석)을 진행한다.

In [13]:
sp.posthoc_conover(complete_episode_day_interval,
                   val_col='avg_complete_episode_intervals',
                   group_col='segmentation',
                   p_adjust='holm')

,냉담,일시적,핵심
냉담,1.000000e+00,5.369305e-26,7.683722e-05
일시적,5.369305e-26,1.000000e+00,8.814066e-30
핵심,7.683722e-05,8.814066e-30,1.000000e+00


* 두 세그먼트에 대한 각 p-value가 모두 매우 작으므로, 모든 사용자 세그먼트의 중위수가 각각 유의미한 차이를 가진다고 말할 수 있다.

### **문제점 발견**

* 정규성을 만족하지 못해 ANOVA 대신 Kruskal-Wallis H Test를 사용했다.
* ANOVA는 집단 간 평균을 비교하지만, Kruskal-Wallis H Test는 집단 간 중위수를 비교한다.
* 따라서 위 분석의 결과는 모든 사용자 세그먼트의 평균이 각각 유의미한 차이를 가지는 게 아닌, 중위수가 각각 유의미한 차이를 가진다는 것이다.
* 그래서 사용자 세그먼트별 `complete.episode` day interval의 중위수를 계산해보았다.

In [14]:
# 사용자 세그먼트별 complete.episode day interval 중위수

print("[사용자 세그먼트별 complete.episode day interval 중위수]\n")
print(f"핵심 사용자: {round(np.median(complete_episode_day_interval_core), 2)}일")
print(f"일시적 사용자: {round(np.median(complete_episode_day_interval_temp), 2)}일")
print(f"냉담 사용자: {round(np.median(complete_episode_day_interval_cold), 2)}일")

[사용자 세그먼트별 complete.episode day interval 중위수]

핵심 사용자: 3.2일
일시적 사용자: 4.78일
냉담 사용자: 3.0일


* 계산 결과, 중위수 크기가 평균처럼 냉담 > 일시적 > 핵심 순이 아닌 **일시적 > 핵심 > 냉담 순**으로, 냉담 사용자의 중위수가 가장 적게 나왔다.

* 우리가 ANOVA로 증명하고자 했던 가설은 다음과 같았다.
> 가설: `complete.episode`(레슨 완료) 액션이 이루어진 날(day)들 사이의 간격이 핵심 사용자일수록 작고, 냉담 사용자일수록 클 것이다.
* 따라서 위 분석에서 사용자 세그먼트 간 `complete.episode` day interval 중위수의 차이가 있다고 결과가 나왔어도, 위 가설을 증명할 수는 없는 결과이다.

* 사용자 세그먼트별 complete.episode day interval의 평균과 중위수 결과를 보고, 이상치가 원인일 것이라는 생각이 들었다.
* 따라서 사용자 세그먼트별로 데이터의 최솟값과 최댓값을 살펴보았다.

In [15]:
# 사용자 세그먼트별 complete.episode day interval 최솟값과 최댓값

print("[사용자 세그먼트별 complete.episode day interval 최솟값과 최댓값]\n")
print(f"핵심 사용자 - 최솟값: {round(np.min(complete_episode_day_interval_core), 2)}일, 최댓값: {round(np.max(complete_episode_day_interval_core), 2)}일")
print(f"일시적 사용자 - 최솟값: {round(np.min(complete_episode_day_interval_temp), 2)}일, 최댓값: {round(np.max(complete_episode_day_interval_temp), 2)}일")
print(f"냉담 사용자 - 최솟값: {round(np.min(complete_episode_day_interval_cold), 2)}일, 최댓값: {round(np.max(complete_episode_day_interval_cold), 2)}일")

[사용자 세그먼트별 complete.episode day interval 최솟값과 최댓값]

핵심 사용자 - 최솟값: 1.0일, 최댓값: 28.23일
일시적 사용자 - 최솟값: 1.0일, 최댓값: 149.0일
냉담 사용자 - 최솟값: 1.0일, 최댓값: 624.0일


* 최솟값은 1일로 동일했지만, 최댓값은 크게 차이가 났다.
* 특히 냉담 사용자의 최댓값인 624일을 보면, 해당 유저는 레슨을 완료한 날 이후로 623일을 레슨을 완료하지 않다가 그 다음 날 레슨을 완료한 것이다.
* 이러한 유저는 흔치 않을 것이고, 따라서 이러한 유저가 평균에 같이 계산된다면 정확한 결과가 나오지 않을 것이다.
* 따라서 이상치인 유저들을 계산해 해당 유저들을 제거 후 다시 분석을 진행하였다.

## **두번째 시도**

### **이상치 처리**

In [16]:
# 이상치 제거 전 사용자 세그먼트별 사용자 수

print("[사용자 세그먼트별 사용자 수]\n")
print(f"핵심 사용자: {complete_episode_day_interval_core.size}명")
print(f"일시적 사용자: {complete_episode_day_interval_temp.size}명")
print(f"냉담 사용자: {complete_episode_day_interval_cold.size}명")

[사용자 세그먼트별 사용자 수]

핵심 사용자: 1200명
일시적 사용자: 5602명
냉담 사용자: 3375명


In [17]:
# 사용자 세그먼트별 이상치 제거

series_names = ['complete_episode_day_interval_core', 'complete_episode_day_interval_temp', 'complete_episode_day_interval_cold']

def remove_outliers(series_names):
    for name in series_names:
        if name in globals():
            series = globals()[name]
            q1 = series.quantile(0.25)
            q3 = series.quantile(0.75)
            iqr = q3 - q1
            filtered_series = series[(series >= q1 - (1.5*iqr)) & (series <= q3 + (1.5*iqr))]
            globals()[name] = filtered_series
        else:
            print(f"{name}는 존재하지 않는 Series 이름입니다.")

remove_outliers(series_names)

In [18]:
# 이상치 제거 후 사용자 세그먼트별 사용자 수

print("[사용자 세그먼트별 사용자 수]\n")
print(f"핵심 사용자: {complete_episode_day_interval_core.size}명")
print(f"일시적 사용자: {complete_episode_day_interval_temp.size}명")
print(f"냉담 사용자: {complete_episode_day_interval_cold.size}명")

[사용자 세그먼트별 사용자 수]

핵심 사용자: 1123명
일시적 사용자: 5144명
냉담 사용자: 2956명


* 총 1200명의 핵심 사용자 세그먼트에서 이상치인 유저 수는 77명(약 6%)이고,  
총 5602명의 일시적 사용자 세그먼트에서 이상치인 유저 수는 458명(약 8%)이고,  
총 3375명의 냉담 사용자 세그먼트에서 이상치인 유저 수는 419명(약 12%)이다.

* 이상치를 제거해도 냉담 사용자의 88%, 일시적 사용자의 92%, 핵심 사용자의 94%에 대한 평균 `complete.episode` day interval 차이를 설명할 수 있다.
* 따라서 위와 같이 이상치를 제거한 후 분석을 진행한다.

In [19]:
# 사용자 세그먼트별 평균 complete.episode day interval

print("[사용자 세그먼트별 평균 complete.episode day interval]\n")
print(f"핵심 사용자: {round(np.mean(complete_episode_day_interval_core), 2)}일")
print(f"일시적 사용자: {round(np.mean(complete_episode_day_interval_temp), 2)}일")
print(f"냉담 사용자: {round(np.mean(complete_episode_day_interval_cold), 2)}일")

[사용자 세그먼트별 평균 complete.episode day interval]

핵심 사용자: 3.52일
일시적 사용자: 5.64일
냉담 사용자: 6.52일


* 이상치를 제거하기 전보다 사용자 세그먼트 간 평균 간격이 많이 좁아졌다.

### **ANOVA 진행 전 데이터 사전 검정**

* ANOVA를 진행하기 전 데이터에 대한 사전 검정이 필요하다.
* 첫째로 각 세그먼트의 데이터가 정규성을 만족하고, 둘째로 세 세그먼트의 데이터가 등분산성을 만족해야 ANOVA를 진행할 수 있다.

#### **1. 정규성 검정**

* 사용자 세그먼트별 사용자 수를 확인하면 핵심 사용자와 냉담 사용자는 5000명 미만이고, 일시적 사용자는 5000명 이상인 것을 확인할 수 있다.
* 따라서 핵심 사용자와 냉담 사용자는 Shapiro-Wilk test로, 일시적 사용자는 Anderson-Darling test로 정규성 검정을 진행한다.

In [20]:
# 핵심 사용자 세그먼트 정규성 검정

print(stats.shapiro(complete_episode_day_interval_core))

ShapiroResult(statistic=np.float64(0.9203012724991974), pvalue=np.float64(6.921560402113618e-24))


* p-value가 6.921560402113618e-24로 매우 작으므로, 핵심 사용자 세그먼트의 데이터는 정규성을 만족하지 못한다고 할 수 있다.

In [21]:
# 일시적 사용자 세그먼트 정규성 검정

print(stats.anderson(complete_episode_day_interval_temp, dist='norm'))

AndersonResult(statistic=np.float64(207.61948404548275), critical_values=array([0.576, 0.655, 0.786, 0.917, 1.091]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=np.float64(5.643899085309008), scale=np.float64(4.25357351741433))
 success: True
 message: '`anderson` successfully fit the distribution to the data.')


* 검정통계량 값은 207.61948404548275로, 유의수준이 1%일 때의 임계값인 1.091보다도 매우 크다.
* 따라서 일시적 사용자 세그먼트의 데이터가 정규분포에서 추출되었다는 귀무가설은 기각된다.
* 즉 일시적 사용자 세그먼트의 데이터는 정규성을 만족하지 못한다고 할 수 있다.

In [22]:
# 냉담 사용자 세그먼트 정규성 검정

print(stats.shapiro(complete_episode_day_interval_cold))

ShapiroResult(statistic=np.float64(0.6869691580442279), pvalue=np.float64(3.7926396530203758e-59))


* p-value가 3.7926396530203758e-59로 매우 작으므로, 냉담 사용자 세그먼트의 데이터는 정규성을 만족하지 못한다고 할 수 있다.

* test 결과, 사용자 세그먼트별로 각 사용자의 평균 `complete.episode` day interval들은 **정규성을 만족하지 못한다**.
* 하지만 위에서 사용한 test는 귀무가설이 채택되기 상당히 까다로운 test이다.
* 이로 인해 **실무에서는 왜도의 절대값이 2보다 작고, 첨도가 7보다 작으면 정규성을 만족한다**고 본다. (표본 수가 300 이상일 경우)  
참고: [Statistical notes for clinical researchers: assessing normal distribution (2) using skewness and kurtosis](https://pmc.ncbi.nlm.nih.gov/articles/PMC3591587/)

##### **Practical Approach: 왜도와 첨도 이용**

In [23]:
# 사용자 세그먼트별 데이터의 왜도의 절대값 계산

print("[사용자 세그먼트별 데이터의 왜도의 절대값]\n")
print(f"핵심 사용자: {round(abs(stats.skew(complete_episode_day_interval_core, bias=False)), 2)}")
print(f"일시적 사용자: {round(abs(stats.skew(complete_episode_day_interval_temp, bias=False)), 2)}")
print(f"냉담 사용자: {round(abs(stats.skew(complete_episode_day_interval_cold, bias=False)), 2)}")

[사용자 세그먼트별 데이터의 왜도의 절대값]

핵심 사용자: 0.95
일시적 사용자: 1.21
냉담 사용자: 1.87


* 왜도의 절대값이 모두 2보다 작다.

In [24]:
# (왜도의 절대값이 모두 2보다 크면 무용지물이지만) 사용자 세그먼트별 데이터의 첨도 계산

print("[사용자 세그먼트별 데이터의 첨도]\n")
print(f"핵심 사용자: {round(stats.kurtosis(complete_episode_day_interval_core, fisher=True), 2)}")
print(f"일시적 사용자: {round(stats.kurtosis(complete_episode_day_interval_temp, fisher=True), 2)}")
print(f"냉담 사용자: {round(stats.kurtosis(complete_episode_day_interval_cold, fisher=True), 2)}")

[사용자 세그먼트별 데이터의 첨도]

핵심 사용자: 0.28
일시적 사용자: 0.8
냉담 사용자: 2.55


* 첨도가 모두 7보다 작다.

* 왜도와 첨도를 계산한 결과, 사용자 세그먼트별 데이터들은 정규성을 만족한다고 볼 수 있다.
* 따라서 다음으로 등분산성을 만족하는지 알아본다.

#### **2. 등분산성 검정**

In [25]:
# Levene 검정

print(stats.levene(complete_episode_day_interval_core, complete_episode_day_interval_temp, complete_episode_day_interval_cold))

LeveneResult(statistic=np.float64(263.87078624985634), pvalue=np.float64(3.6481648190671024e-112))


* p-value가 3.6481648190671024e-112로 매우 작으므로, 세 세그먼트의 데이터는 등분산성을 만족하지 못한다고 할 수 있다.

In [26]:
# Bartlett 검정

print(stats.bartlett(complete_episode_day_interval_core, complete_episode_day_interval_temp, complete_episode_day_interval_cold))

BartlettResult(statistic=np.float64(3540.336282854698), pvalue=np.float64(0.0))


* p-value가 0이므로, 세 세그먼트의 데이터는 등분산성을 만족하지 못한다고 할 수 있다.

* 따라서 이분산이 가정된 ANOVA를 진행한다.

### **이분산이 가정된 One-way ANOVA**

* $H_0$: 모든 집단의 평균이 같다.
* $H_1$: 한 집단이라도 평균이 같지 않다.

In [27]:
# Welch's ANOVA

print(welch_anova(data=complete_episode_day_interval, dv='avg_complete_episode_intervals', between='segmentation'))

         Source  ddof1        ddof2           F          p-unc       np2
0  segmentation      2  5789.521626  445.773774  8.636939e-181  0.038527


* 여기서 p-unc 값이 p-value이다.
* p-value가 8.636939e-181로 매우 작으므로 귀무가설은 기각된다.
* 즉 한 사용자 세그먼트라도 평균이 같지 않다고 할 수 있다.

### **다중비교(사후분석)**

* 우리는 모든 사용자 세그먼트의 평균이 각각 유의미한 차이를 가지는지 알고 싶기 때문에 다중비교(사후분석)을 진행한다.

In [28]:
# Tukey 사후검정

tukey_result = pairwise_tukeyhsd(endog=complete_episode_day_interval['avg_complete_episode_intervals'], groups=complete_episode_day_interval['segmentation'])
print(tukey_result)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj  lower    upper   reject
-----------------------------------------------------
    냉담    일시적  -9.5335   0.0 -10.7928  -8.2741   True
    냉담     핵심 -13.2489   0.0 -15.1913 -11.3065   True
   일시적     핵심  -3.7155   0.0  -5.5538  -1.8771   True
-----------------------------------------------------


* 두 세그먼트에 대한 각 reject값이 모두 True이므로, 모든 사용자 세그먼트의 평균이 각각 유의미한 차이를 가진다고 말할 수 있다.

## **결론**

In [29]:
# 사용자 세그먼트별 평균 complete.episode day interval

print("[사용자 세그먼트별 평균 complete.episode day interval]\n")
print(f"핵심 사용자: {round(np.mean(complete_episode_day_interval_core), 2)}일")
print(f"일시적 사용자: {round(np.mean(complete_episode_day_interval_temp), 2)}일")
print(f"냉담 사용자: {round(np.mean(complete_episode_day_interval_cold), 2)}일")

[사용자 세그먼트별 평균 complete.episode day interval]

핵심 사용자: 3.52일
일시적 사용자: 5.64일
냉담 사용자: 6.52일


* 사용자 세그먼트별 평균 `complete.episode` day interval은 위와 같다.
* ANOVA를 한 결과, 모든 사용자 세그먼트의 평균이 각각 유의미한 차이를 가진다고 말할 수 있다.
* 이를 통해 우리가 증명하려던 아래의 가설도 증명이 되었다.
> 가설: `complete.episode`(레슨 완료) 액션이 이루어진 날(day)들 사이의 간격이 핵심 사용자일수록 작고, 냉담 사용자일수록 클 것이다.

### **향후 진행 방향**

In [30]:
# 사용자 세그먼트별 평균 complete.episode day interval

print("[사용자 세그먼트별 평균 complete.episode day interval]\n")
print(f"핵심 사용자: {round(np.mean(complete_episode_day_interval_core), 2)}일")
print(f"일시적 사용자: {round(np.mean(complete_episode_day_interval_temp), 2)}일")
print(f"냉담 사용자: {round(np.mean(complete_episode_day_interval_cold), 2)}일")

[사용자 세그먼트별 평균 complete.episode day interval]

핵심 사용자: 3.52일
일시적 사용자: 5.64일
냉담 사용자: 6.52일


* 사용자 세그먼트별 평균 `complete.episode` day interval 값을 통해 마케팅 전략을 수립한다.
* 예를 들어 Reminder는 3일 이상 레슨을 완료하지 않을 시 보내고,  
게임 Item처럼 학습에 유용하게 사용할 수 있는 Item은 일주일(7일) 이상 레슨을 완료하지 않을 시 지급하는 전략을 생각해볼 수 있다.